In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer #for missing value
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('dataset/covid_toy.csv')
df.sample(6)

,age,gender,fever,cough,city,has_covid
94,79,Male,NaN,Strong,Kolkata,Yes
15,70,Male,103.0,Strong,Kolkata,Yes
18,64,Female,98.0,Mild,Bangalore,Yes
38,49,Female,101.0,Mild,Delhi,Yes
98,5,Female,98.0,Strong,Mumbai,No
70,68,Female,101.0,Strong,Delhi,No


In [6]:
df #cough is ordinal gender is OHE

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No
...,...,...,...,...,...,...
95,12,Female,104.0,Mild,Bangalore,No
96,51,Female,101.0,Strong,Kolkata,Yes
97,20,Female,101.0,Mild,Bangalore,No
98,5,Female,98.0,Strong,Mumbai,No


In [5]:
df['city']

0       Kolkata
1         Delhi
2         Delhi
3       Kolkata
4        Mumbai
        ...    
95    Bangalore
96      Kolkata
97    Bangalore
98       Mumbai
99      Kolkata
Name: city, Length: 100, dtype: object

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

In [8]:
X_train

,age,gender,fever,cough,city
19,42,Female,NaN,Strong,Bangalore
38,49,Female,101.0,Mild,Delhi
51,11,Female,100.0,Strong,Kolkata
11,65,Female,98.0,Mild,Mumbai
27,33,Female,102.0,Strong,Delhi
...,...,...,...,...,...
99,10,Female,98.0,Strong,Kolkata
72,83,Female,101.0,Mild,Kolkata
37,55,Male,100.0,Mild,Kolkata
33,26,Female,98.0,Mild,Kolkata


# w/o column transformer

In [12]:
X_train['fever']  # has NaN

19      NaN
38    101.0
51    100.0
11     98.0
27    102.0
      ...  
99     98.0
72    101.0
37    100.0
33     98.0
14    104.0
Name: fever, Length: 80, dtype: float64

In [13]:
# imputer replace missing by its data mean

# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])
                                 
X_train_fever.shape

(80, 1)

In [14]:
X_train

,age,gender,fever,cough,city
19,42,Female,NaN,Strong,Bangalore
38,49,Female,101.0,Mild,Delhi
51,11,Female,100.0,Strong,Kolkata
11,65,Female,98.0,Mild,Mumbai
27,33,Female,102.0,Strong,Delhi
...,...,...,...,...,...
99,10,Female,98.0,Strong,Kolkata
72,83,Female,101.0,Mild,Kolkata
37,55,Male,100.0,Mild,Kolkata
33,26,Female,98.0,Mild,Kolkata


In [15]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [16]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

C:\Users\ALOK SINGH\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\ALOK SINGH\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 4)

In [17]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [18]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

# column transformer

In [19]:
from sklearn.compose import ColumnTransformer

In [22]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')


transformer.fit_transform(X_train).shape

C:\Users\ALOK SINGH\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 7)

In [23]:
transformer.transform(X_test).shape

(20, 7)